# Dataset Transform
- Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
- Refernce: [Torch Transform](https://pytorch.org/vision/stable/transforms.html)



In [ ]:
'''
On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
On Tensors
----------
LinearTransformation, Normalize, RandomErasing
Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
Generic
-------
Use Lambda 
Custom
------
Write own class
Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

In [5]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [6]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        #data loading
        xy = np.loadtxt('./data/wine.csv', delimiter=',', dtype=np.float32, skiprows=1) #skiprows=1 skip header row
        self.n_samples = xy.shape[0]

        self.x = xy[:, 1:]
        self.y = xy[:, [0]] # [0]  : (n_samples, 1)
        
        self.transform = transform


    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples

class ToTensor:
    def __call__(self, sample):
        #callable object
        inputs, target = sample
        return torch.from_numpy(inputs), torch.from_numpy(target)


#transform to convert X, y into Tensor
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, label = first_data
print(type(features), type(label)) #<class 'torch.Tensor'> <class 'torch.Tensor'>
print(features, label)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [7]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

#Compose of List of Transforms
composed_transforms = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed_transforms)
first_data = dataset[0]
features, label = first_data
print(features, label)

tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])
